## Create Default Folio

In [ ]:
# Bootstrapping
import logging

import pandas as pd

from app.bootstrap import reload_config
from mock.folio_setup import ensure_folio_exists

config = reload_config()
ensure_folio_exists()
logging.shutdown()

## Read Folio Tickers

In [ ]:
import logging

import pandas as pd

from app.bootstrap import reload_config

config = reload_config()
df = pd.read_excel(config.folio_path, config.tickers_sheet())
print(df)
logging.shutdown()

## Read Folio Transactions

In [ ]:
import logging

import pandas as pd

from app.bootstrap import reload_config

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.float_format", "{:.10f}".format)  # No scientific notation
config = reload_config()
txns = pd.read_excel(config.folio_path, config.transactions_sheet())
txns = txns.drop(columns="Description", errors="ignore")
print(txns)
logging.shutdown()

## Read Folio FX Rates

In [ ]:
import logging

import pandas as pd

from app.bootstrap import reload_config

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", None)
config = reload_config()
fx = pd.read_excel(config.folio_path, config.forex_sheet())
print(fx)
logging.shutdown()

## Nuke Local Data Directory

In [ ]:
import logging
import shutil

from app.bootstrap import reload_config

config = reload_config()
data_dir = config.project_root / "data"
delete: bool = True  # Set to True to delete existing data directory
# Check if the directory exists
if data_dir.exists():
    if delete:
        try:
            # Use shutil.rmtree to delete the directory and its contents
            shutil.rmtree(data_dir)
            print(f"Deleted directory: {data_dir}")
        except PermissionError as e:
            print(f"Permission denied: {e}")
        except FileNotFoundError as e:
            print(f"Directory not found: {e}")
else:
    print(f"Directory does not exist: {data_dir}")

logging.shutdown()

## Reorder Folio Columns

**New Order**: SettleDate, TxnDate, Action, Amount, $, Price, Units, Fee, Account, (blank), Ticker, (other columns)


In [ ]:
import logging

import pandas as pd

from app.bootstrap import reload_config
from exporters.transaction_exporter import reorder_folio_columns
from utils.backup import rolling_backup


def reorder_folio() -> None:
    """Reorder the folio columns."""
    config = reload_config()
    if not config.folio_path.exists():
        msg = f"Excel file not found: {config.folio_path}"
        print(msg)
        return

    try:
        excel_df = pd.read_excel(
            config.folio_path,
            sheet_name=config.transactions_sheet(),
            engine="openpyxl",
        )
        reordered_df = reorder_folio_columns(excel_df)
        rolling_backup(config.folio_path)
        reordered_df.to_excel(
            config.folio_path,
            index=False,
            sheet_name=config.transactions_sheet(),
            engine="openpyxl",
        )
    except ValueError as e:
        msg = f"Error reading sheet '{config.transactions_sheet()}': {e}"
        print(msg)
        return


reorder_folio()
logging.shutdown()